<div align="center">

# RIO Airbnb Project - Reviews Clean
**Latest Update:** *15th March 2023*

</div>

---

In [1]:
import sys, os
sys.path.append(os.path.abspath("../"))

import pandas as pd

In [2]:
reviews_df = pd.read_csv("../dataset/reviews/reviews.csv", index_col=False)

In [3]:
# Import raw CSV of latest version
newest_listing_csv = os.listdir("../dataset/listings/")[-1]
listings_df = pd.read_csv(f"../dataset/listings/{newest_listing_csv}")

newest_listing_csv

'listings_cleaned_20230315.csv'

In [3]:
# Import zipped CSV of latest version
newest_listing_zip = os.listdir("../dataset_zipped/listings/")[-1]
listings_df = pd.read_csv(f"../dataset_zipped/listings/{newest_listing_zip}")

newest_listing_zip

'listings_cleaned_20230303.zip'

In [4]:
listings_df['id'].head()

0    183319.0
1     51307.0
2    184314.0
3     51498.0
4       109.0
Name: id, dtype: float64

In [5]:
reviews_filtered = reviews_df[reviews_df['listing_id'].isin(listings_df['id'].unique())]

len(reviews_filtered)

1557391

In [38]:
reviews_with_listing_df = {
  'review': [],
  'listing_id': [],
  'price': []
}

for rev in reviews_filtered.index.to_list():
  reviews_with_listing_df['review'].append(reviews_filtered['comments'][rev])
  
  reviews_with_listing_df['listing_id'].append(reviews_filtered['listing_id'][rev])
  
  reviews_with_listing_df['price'].append(
    list(listings_df[listings_df['id'] == reviews_filtered['listing_id'][rev]]['price'])[0]
  )

reviews_with_listing_df = pd.DataFrame(reviews_with_listing_df)

In [40]:
cols = reviews_with_listing_df.columns.to_list()[::-1]
reviews_with_listing_df = reviews_with_listing_df[cols]
reviews_with_listing_df

,price,listing_id,review
0,115.0,109,The host canceled my reservation the day befor...
1,115.0,109,Me and two friends stayed for four and a half ...
2,93.0,2708,i had a wonderful stay. Everything from start ...
3,152.0,183319,"very nice and cozy apartment, well located in ..."
4,152.0,183319,We loved the experience. The location couldn't...
...,...,...,...
1549980,209.0,19473980,Great location! Would recommend it to anyone w...
1549981,209.0,19473980,What a perfect place to explore Balboa Peninsu...
1549982,209.0,19473980,Great spot in an even better location! Clean a...
1549983,209.0,19473980,The condo was comfortable and clean. It’s very...


In [45]:
listings_df['reviews_available'] = [
  reviews_with_listing_df[reviews_with_listing_df['listing_id'] == id]['review'].count()
  for id in listings_df['id']
]

listings_df[['id', 'reviews_available', 'number_of_reviews']].head(10)

,id,reviews_available,number_of_reviews
0,183319.0,3,3
1,51307.0,138,138
2,184314.0,30,30
3,51498.0,378,378
4,109.0,2,2
5,185536.0,13,13
6,2708.0,37,37
7,51546.0,188,188
8,185557.0,12,12
9,2732.0,24,24


In [50]:
new_df = listings_df.copy()

for i in range(len(listings_df['reviews_available'])):
  if listings_df['reviews_available'][i] != listings_df['number_of_reviews'][i]:
    try:
      new_df = new_df[new_df['reviews_available'] != new_df['number_of_reviews'][i]]
    except KeyError:
      pass

print(f'before: {listings_df.id.count()}')
print(f'before: {new_df.id.count()}')

before: 45008
before: 20872
